In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
general_data =pd.read_csv('/kaggle/input/hr-analytics-case-study/general_data.csv')
general_data.head()


In [ ]:
employee_survey_data= pd.read_csv('/kaggle/input/hr-analytics-case-study/employee_survey_data.csv')
employee_survey_data.head()           



In [ ]:
manager_survey_data = pd.read_csv('/kaggle/input/hr-analytics-case-study/manager_survey_data.csv')
manager_survey_data.head()

In [ ]:
main_data = general_data.join([manager_survey_data.drop('EmployeeID',axis=1),
                             employee_survey_data.drop('EmployeeID',axis=1)])
main_data.head()
main_data.drop('EmployeeID',axis=1,inplace=True)
main_data

In [ ]:
main_data.info()

In [ ]:
main_data.nunique()
#Categorical=['BusinessTravel','Department','Education ','EducationField ','EmployeeCount,'Gender',                        
#JobLevel , JobRole,MaritalStatus',NumCompaniesWorked ,Over18,PercentSalaryHike,StandardHours ,                1
#StockOptionLevel ,TrainingTimesLastYear,JobInvolvement, PerformanceRating ,EnvironmentSatisfaction       4
#JobSatisfaction,WorkLifeBalance ]
#Continous= ['Age','DistanceFromHome','MonthlyIncome ','TotalWorkingYears ','YearsAtCompany',]

In [ ]:
main_data.isnull().sum()

In [ ]:
main_data['Attrition'].value_counts().plot(kind='bar')


In [ ]:
main_data['Attrition'].value_counts()

In [ ]:
main_data['NumCompaniesWorked'].fillna(main_data['NumCompaniesWorked'].mode()[0],inplace= True)
main_data['EnvironmentSatisfaction'].fillna(main_data['EnvironmentSatisfaction'].mean(),inplace= True)

main_data['JobSatisfaction'].fillna(main_data['JobSatisfaction'].mean(), inplace=True)
main_data['WorkLifeBalance'].fillna(main_data['WorkLifeBalance'].mean(), inplace=True)
main_data.isnull().sum()

In [ ]:
import pandas as pd
def PlotBarCharts (inpData, colsToPlot):
    %matplotlib inline
    
    import matplotlib.pyplot as plt
    
    
    fig,subplot= plt.subplots(nrows= 1, ncols=len(colsToPlot), figsize=(10,5))
    fig.suptitle('Bar Charts Of'+ str(colsToPlot))
    
    for colName, i in zip(colsToPlot, range(len(colsToPlot))):
        inpData.groupby(colsToPlot).size().plot(kind='bar',ax = subplot[i])

In [ ]:
PlotBarCharts(inpData=main_data, colsToPlot=['BusinessTravel','Department','EducationField'])
                                             

In [ ]:
def FunctionChisq(inpData, TargetVariable, CategoricalVariablesList):
    from scipy.stats import chi2_contingency
    
    # Creating an empty list of final selected predictors
    SelectedPredictors=[]

    for predictor in CategoricalVariablesList:
        CrossTabResult=pd.crosstab(index=inpData[TargetVariable], columns=inpData[predictor])
        ChiSqResult = chi2_contingency(CrossTabResult)
        
        # If the ChiSq P-Value is <0.05, that means we reject H0
        if (ChiSqResult[1] < 0.05):
            print(predictor, 'is correlated with', TargetVariable, '| P-Value:', ChiSqResult[1])
            SelectedPredictors.append(predictor)
        else:
            print(predictor, 'is NOT correlated with', TargetVariable, '| P-Value:', ChiSqResult[1])        
            
    return(SelectedPredictors)

In [ ]:
CategoricalVariables=['BusinessTravel','Department','EducationField','EmployeeCount',
                      'Gender','JobLevel' , 'JobRole','MaritalStatus','NumCompaniesWorked',
                      'Over18','PercentSalaryHike','StandardHours','StockOptionLevel',
                      'TrainingTimesLastYear','JobInvolvement', 'PerformanceRating' ,
                      'EnvironmentSatisfaction',
                      'JobSatisfaction','WorkLifeBalance' ]

# Calling the function
FunctionChisq(inpData=main_data, 
              TargetVariable='Attrition',
              CategoricalVariablesList= CategoricalVariables)

In [ ]:
#'EmployeeCount''Gender','JobLevel' 'Over18','PercentSalaryHike','StandardHours','StockOptionLevel'
#'PerformanceRating'

main_data.drop(['EmployeeCount','Gender','JobLevel', 'Over18',
                'PercentSalaryHike','StandardHours','StockOptionLevel',
                'PerformanceRating'],axis=1, inplace= True)
main_data.head()

In [ ]:
def FunctionAnova(inpData, TargetVariable, ContinousVariable):
    from scipy.stats import f_oneway
    
    SelectedPredictors=[]
    
    for predictor in ContinousVariable:
        CategoryList= inpData.groupby(TargetVariable)[predictor].apply(list)
        AnnovaResults = f_oneway(*CategoryList)
        
        
        if (AnnovaResults[1] < 0.05):
            print(predictor, 'is corelated to the ',TargetVariable ,'|p Value', AnnovaResults[1])
            SelectedPredictors.append(predictor)
        else:
            print(predictor, 'is not corelated to the ',TargetVariable ,'|p Value', AnnovaResults[1])
            
    return(SelectedPredictors)
    

In [ ]:
ContinuousData=['Age','DistanceFromHome','MonthlyIncome','TotalWorkingYears','YearsAtCompany']

# Calling the function
FunctionAnova(inpData=main_data, 
              TargetVariable='Attrition',
              ContinousVariable= ContinuousData)

In [ ]:
main_data.drop(['DistanceFromHome','TotalWorkingYears'], axis= 1, inplace=True)

In [ ]:
main_data.head()

In [ ]:
main_data.hist(['Age', 'MonthlyIncome', 'YearsAtCompany'],figsize=(18,6))

In [ ]:
main_data.head()

In [ ]:
main_data = pd.get_dummies(main_data, columns=['BusinessTravel', 'Department', 'EducationField', 
                                               'JobRole', 'MaritalStatus'])
main_data.nunique()

In [ ]:
Predictors=['Age','Education','MonthlyIncome','NumCompaniesWorked','TrainingTimesLastYear',
            'YearsAtCompany', 'YearsSinceLastPromotion','YearsWithCurrManager','JobInvolvement',
            'EnvironmentSatisfaction','JobSatisfaction','WorkLifeBalance','BusinessTravel_Non-Travel',
            'BusinessTravel_Travel_Frequently','BusinessTravel_Travel_Rarely','Department_Human Resources',
'Department_Research & Development','Department_Sales','EducationField_Human Resources',
'EducationField_Life Sciences','EducationField_Marketing','EducationField_Medical',                  
'EducationField_Other','EducationField_Technical Degree',         
'JobRole_Healthcare Representative',       
'JobRole_Human Resources','JobRole_Laboratory Technician',          
'JobRole_Manager','JobRole_Manufacturing Director',
'JobRole_Research Director','JobRole_Research Scientist',              
'JobRole_Sales Executive','JobRole_Sales Representative',           
'MaritalStatus_Divorced','MaritalStatus_Married', 'MaritalStatus_Single'  ]

TargetVariable ='Attrition'
X= main_data[Predictors].values
y=main_data[TargetVariable].values

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')
X_train , X_test , y_train, y_test = train_test_split(X , y , train_size = 0.8 , random_state = 2)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 6), dpi=80)
features_corr = pd.DataFrame(X_train).corr()
sns.heatmap(features_corr)


In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression() # Changing Line in the code

model.fit(X_train , y_train)
y_pred = model.predict(X_test)

accuracy_score(y_test, y_pred)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_train , y_train)
y_pred = model.predict(X_test)

accuracy_score(y_test, y_pred)

In [ ]:
df_predicted=pd.DataFrame()
df_predicted['Actual']=y_test
df_predicted['Predicted']=y_pred
df_predicted